### Demonstration of running tensorflow transformers model 

In [ ]:
from bertviz import head_view
import torch
from transformers import BertTokenizer, TFBertForPreTraining, utils

# Note that tensorflow must be installed (not in requirements.txt)
import tensorflow as tf


In [ ]:
def show_head_view_tf(model, tokenizer, sentence_a, sentence_b=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='tf', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        sentence_b_start = token_type_ids[0].numpy().tolist().index(1)
    else:
        attention = model(input_ids)[-1]
        sentence_b_start = None
    
    # Convert attention from TF tensors to torch tensors
    attention = [torch.from_numpy(layer_attn.numpy()) for layer_attn in attention]
    
    input_id_list = input_ids[0].numpy().tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start)


In [ ]:
utils.logging.set_verbosity_error()  # Remove line to see warnings
sentence_a = "The cat sat on the mat"
sentence_b = "The cat lay on the rug"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForPreTraining.from_pretrained('bert-base-uncased', output_attentions=True)
show_head_view_tf(model, tokenizer, sentence_a, sentence_b)